<a href="https://colab.research.google.com/github/ramkumardeepak774/Text_Classification_Flair-vs-Fasttext/blob/main/using_flair_to_classify_on_idfc_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 71.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 79.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.4 MB/s eta 0:00:0

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
idfc_train = pd.read_csv("/content/idfc-email-train.csv")
idfc_test = pd.read_csv("/content/idfc-email-test.csv")

In [ ]:
#change the order of column in idfc data
idfc_train = idfc_train .iloc[:,[1,0]]
idfc_test = idfc_test.iloc[:, [1,0]]

In [ ]:
idfc_train.columns  = ['text', 'label']
idfc_test.columns  = ['text', 'label']

In [ ]:
from sklearn.model_selection import train_test_split


In [40]:
train_data = idfc_train
test_data = idfc_test

In [ ]:
mkdir -p data_fst


In [41]:
print(train_data.shape)
print(test_data.shape)


(1440, 2)
(360, 2)


In [83]:
train_data.to_csv("data_fst/train.csv")
test_data.to_csv("data_fst/test.csv")


In [ ]:
import flair

In [84]:
from flair.datasets import ClassificationCorpus,CSVClassificationCorpus
from flair.data import Corpus

In [ ]:
column_name_map = {2:"label_topic",1:"text"}


In [ ]:
data_folder = '/content/data_fst/'


In [144]:
corpus_csv: Corpus = CSVClassificationCorpus(data_folder,column_name_map=column_name_map,skip_header=True, label_type='topic', delimiter=',')


2023-05-11 11:52:12,719 Reading data from /content/data_fst
2023-05-11 11:52:12,721 Train: /content/data_fst/train.csv
2023-05-11 11:52:12,722 Dev: None
2023-05-11 11:52:12,724 Test: /content/data_fst/test.csv


In [145]:
label_dict_csv = corpus_csv.make_label_dictionary(label_type = 'topic')


2023-05-11 11:52:17,967 Computing label dictionary. Progress:


1296it [00:03, 351.09it/s]

2023-05-11 11:52:21,664 Dictionary created for label 'topic' with 7 values: Debit Card Offers (seen 223 times), Fund Transfer (seen 221 times), Matured Loan Closure (seen 218 times), Card Application Status (seen 214 times), Other Bank Charges (seen 214 times), Application Rejected (seen 206 times)


In [89]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [152]:
from flair.embeddings import SentenceTransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.embeddings import TransformerDocumentEmbeddings

In [154]:
# document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

document_embeddings = SentenceTransformerDocumentEmbeddings('sentence-transformers/bert-base-nli-mean-tokens')
# document_embeddings = SentenceTransformerDocumentEmbeddings('sentence-transformers/distiluse-base-multilingual-cased-v2')

# document_embeddings = SentenceTransformerDocumentEmbeddings('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# document_embeddings = SentenceTransformerDocumentEmbeddings('sentence-transformers/all-MiniLM-L6-v2')

# document_embeddings = SentenceTransformerDocumentEmbeddings('sentence-transformers/all-mpnet-base-v2')




In [155]:
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict_csv, multi_label=False, label_type="topic")


In [156]:
trainer = ModelTrainer(classifier, corpus_csv)


In [157]:
import torch
torch.cuda.empty_cache()

In [158]:
trainer.train(data_folder,
              learning_rate=0.05,
              mini_batch_size=10,
              max_epochs=5)

2023-05-11 11:58:13,517 ----------------------------------------------------------------------------------------------------
2023-05-11 11:58:13,523 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out

100%|██████████| 15/15 [00:02<00:00,  6.35it/s]

2023-05-11 11:59:05,212 Evaluating as a multi-label problem: False
2023-05-11 11:59:05,234 DEV : loss 10.002354621887207 - f1-score (micro avg)  0.1806


2023-05-11 11:59:05,628 BAD EPOCHS (no improvement): 0
2023-05-11 11:59:05,633 saving best model
2023-05-11 11:59:06,397 ----------------------------------------------------------------------------------------------------
2023-05-11 11:59:11,823 epoch 2 - iter 13/130 - loss 2.91694552 - time (sec): 5.42 - samples/sec: 23.97 - lr: 0.050000
2023-05-11 11:59:17,233 epoch 2 - iter 26/130 - loss 2.43109579 - time (sec): 10.83 - samples/sec: 24.00 - lr: 0.050000
2023-05-11 11:59:22,030 epoch 2 - iter 39/130 - loss 2.20828709 - time (sec): 15.63 - samples/sec: 24.95 - lr: 0.050000
2023-05-11 11:59:25,904 epoch 2 - iter 52/130 - loss 1.96433931 - time (sec): 19.50 - samples/sec: 26.66 - lr: 0.050000
2023-05-11 11:59:30,523 epoch 2 - iter 65/130 - loss 1.75897034 - time (sec): 24.12 - samples/sec: 26.94 - lr: 0.050000
2023-05-11 11:59:35,700 epoch 2 - iter 78/130 - loss 1.58488538 - time (sec): 29.30 - samples/sec: 26.62 - lr: 0.050000
2023-05-11 11:59:41,523 epoch 2 - iter 91/130 - loss 1.4673

100%|██████████| 15/15 [00:02<00:00,  7.25it/s]

...

2023-05-11 12:02:35,119 Evaluating as a multi-label problem: False
2023-05-11 12:02:35,134 DEV : loss 0.5655538439750671 - f1-score (micro avg)  0.7639


2023-05-11 12:02:35,807 BAD EPOCHS (no improvement): 1
2023-05-11 12:02:36,660 ----------------------------------------------------------------------------------------------------


100%|██████████| 36/36 [00:05<00:00,  7.01it/s]

2023-05-11 12:02:43,586 Evaluating as a multi-label problem: False
2023-05-11 12:02:43,602 0.7917	0.7917	0.7917	0.7917
2023-05-11 12:02:43,603 
Results:
- F-score (micro) 0.7917
- F-score (macro) 0.7843
- Accuracy 0.7917

By class:
                         precision    recall  f1-score   support

   Application Rejected     0.6076    0.8000    0.6906        60
          Fund Transfer     0.7761    0.8667    0.8189        60
      Debit Card Offers     0.8788    0.9667    0.9206        60
   Matured Loan Closure     0.9231    1.0000    0.9600        60
     Other Bank Charges     0.9767    0.7000    0.8155        60
Card Application Status     0.6250    0.4167    0.5000        60

               accuracy                         0.7917       360
              macro avg     0.7979    0.7917    0.7843       360
           weighted avg     0.7979    0.7917    0.7843       360

2023-05-11 12:02:43,608 -------------------------------------------------------------------------------------------

{'test_score': 0.7916666666666666,
 'dev_score_history': [0.18055555555555555,
  0.6875,
  0.75,
  0.7847222222222222,
  0.7638888888888888],
 'train_loss_history': [0.4220722925309603,
  1.2709644824082469,
  0.6496576164377692,
  0.5070938423827842,
  0.40006980723069036],
 'dev_loss_history': [10.002354621887207,
  0.6676851511001587,
  0.5106480717658997,
  0.5412343144416809,
  0.5655538439750671]}

In [39]:
new_clf = TextClassifier.load('/content/data_fst/best-model.pt')
